<a href="https://colab.research.google.com/github/ItzWarLockOp/Google-Colab-Scriptz/blob/main/Torrent2Gdrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<center> <img src="https://i.ibb.co/5GDWzfV/Picsart-23-04-29-15-39-46-742.png" height=40px class="icon" alt="icon">

# <b><center> Torrent to Gdrive

___________________________________________

# <b> <center> Mount Gdrive   <img src="https://kstatic.googleusercontent.com/files/e20dbc1db7b6eab22fb2d07df8043f8a74b301dbb8c176e6a1992b2dc229c5ae94b5c260cf549c41712fa40b7e639166c6a8461303a27c2a3b9c55b8ca155ec3" class="icon" height="20px" alt="Icon"> 
# <b> <center> + 
# <b> <Center> Install <img src="https://www.libtorrent.org/img/logo-color-text.png" height="20px" class="icon" alt="Icon">

In [ ]:
#@title <b>↓↓ Run the Following Cell
 
from google.colab import drive
drive.mount("/content/drive")

import IPython.display as display
!pip install --quiet --disable-pip-version-check libtorrent
display.display(display.Markdown("### libtorrent installation complete."))
display.clear_output(wait=True)

___________________________________________

#<b> Enter Path📁 and Magnet🧲

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import libtorrent as lt
import time
from tqdm.auto import tqdm


ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

download_path = "/content/drive/MyDrive" #@param {type:"string"}

params = {
    "save_path": download_path,
    "storage_mode": lt.storage_mode_t(2)
}

link = "magnet:?xt=urn:btih:6E7RHPA" #@param {type:"string"}

if "magnet:" in link:
    handle = lt.add_magnet_uri(ses, link, params)
else:
    params["url"] = link
    handle = ses.add_torrent(params)

print("Downloading metadata...")

while not handle.has_metadata():
    time.sleep(1)

print("Starting", handle.name())

with tqdm(total=handle.status().total_wanted, unit='B', unit_scale=True, desc=handle.name()) as pbar:
    while handle.status().state != lt.torrent_status.seeding:
        s = handle.status()
        state_str = ['queued', 'checking', 'downloading metadata', \
                     'downloading', 'finished', 'seeding', 'allocating']
        pbar.set_postfix({'State': state_str[s.state], 'Seeding': s.num_complete, 'Leeching': s.num_incomplete})
        downloaded = s.total_wanted_done
        pbar.update(downloaded - pbar.n)
        pbar.set_description(f"{handle.name()} - {s.download_rate/1000:.1f} KB/s - {s.progress*100:.2f}%")
        time.sleep(1)

print("Download Completed!")

Starting Peter Pan Wendy (2023) [720p] [WEBRip] [YTS.MX]


Peter Pan Wendy (2023) [720p] [WEBRip] [YTS.MX]:   0%|          | 0.00/998M [00:00<?, ?B/s]

Download Completed!
